In [1]:
from Exercise import Exercise, MarkdownBlock
from config import URL, TOKEN
import json

import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
import pandas as pd
plt.rcParams.update({'font.size': 20})

from sklearn.datasets import load_digits
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


Exercise.URL = URL
Exercise.TOKEN = TOKEN

In [31]:
from sympy import Rational, Symbol, latex, UnevaluatedExpr

u = lambda x : UnevaluatedExpr(x)

# Helper functions
def explain_add(a, b):
    assert(np.shape(a) == np.shape(b))
    rows, columns = np.shape(a)
    return sp.Matrix([[Symbol(f"({latex(u(a[i,j]))} + {latex(u(b[i,j]))})") for j in range(columns)] for i in range(rows)])

def symbolic_matrix(character, rows, columns):
    return sp.Matrix([[Symbol(f"{{{character}}}_{{{i+1}, {j+1}}}") for j in range(columns)] for i in range(rows)])

In [32]:
a = np.arange(4)
b = -1 * np.arange(1, 5)
a = sp.Matrix(a)
b = sp.Matrix(b)

s = "What is $@a + @b?$"

params = {}
params["a"] = a
params["b"] = b
e = Exercise(MarkdownBlock(s, params))

e.display()

s1 = "Yes, $@a + @b = @c = @d$!"

params["c"] = explain_add(a,b)
params["d"] = a + b
e.add_answer(a + b, True, MarkdownBlock(s1, params))
e.evaluate_answer(params["d"])

e.x = symbolic_matrix("a", 4,1)
e.y = symbolic_matrix("b", 4,1)
e.z = explain_add(e.x, e.y)
default_feedback = "Remember the definition of matrix addition: $@x + @y = @z$"
e.add_default_feedback(MarkdownBlock(default_feedback, vars(e)))
e.evaluate_answer(1)

e.write()
# e.publish()

In [111]:
s = "What is $@a \cdot @b$?"

rows = np.random.randint(1, 6)
columns = np.random.randint(1, 6)

params = {}
params["a"] = sp.Matrix(np.random.randint(5, size=rows*columns).reshape((rows,columns)))
params["b"] = sp.Matrix(np.random.randint(5, size=(2+rows)*columns).reshape((columns,rows+2)))

e = Exercise(MarkdownBlock(s, params))

ans = params["a"] * params["b"]
display(ans)
e.add_answer(params["a"] * params["b"], True, "That's right!")


params = {}
params["x"] = symbolic_matrix("a", rows, columns)
params["y"] = symbolic_matrix("b", columns, rows)
params["z"] = explain_multiply(params["x"], params["y"])
f = "Remember the definition of matrix multiplication: $@x \cdot @y = @z$"
e.add_default_feedback(MarkdownBlock(f, params))
e.evaluate_answer(1)


e.display()
e.write()
e.publish()

Matrix([
[ 9, 3, 0, 6, 3],
[ 6, 2, 0, 4, 2],
[12, 4, 0, 8, 4]])

Published succesfully, preview at: https://www.mscthesis.nl/preview?id=b4c60375-387f-462f-b534-7e290df2bd69


In [102]:
def explain_multiply(a, b):
    # #rows in b == #columns in a
#     assert(np.shape(a)[1] == np.shape(b)[0])
    rows = np.shape(a)[0]
    columns = np.shape(b)[1]
    result = np.empty(shape=(rows, columns), dtype=object)
    for i in range(rows):
        row = a[i,:]
        for j in range(columns):
            column = b[:,j]
#             print(row)
#             print(column)
            zipped = zip(row, column)
            mapped = list(map(lambda t: f"{latex(u(t[0]))} \cdot {latex(u(t[1]))}", zipped))
            s = Symbol("") 
            result[i, j] = Symbol(" + ".join(mapped), evaluate=False)
                
    return sp.Matrix(result)

# explain_multiply(np.arange(4).reshape((2,2)), np.arange(4).reshape((2,2)))